In [1]:
import pygame
import itertools
import random
import numpy as np

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
### Parameters ###

H = 900
MAX_SPEED = 3 # unit?
FOLLOWING_TIME = 30
BUFFER = 15
SPAWN_PROBABILITY = 0.02
CAR_WIDTH = 10
CAR_LENGTH = (20,27)
TRUCK_LENGTH = (42,50)
TRUCK_PROBABILITY = 0.1
ACCELERATION = 0.015
DECELERATION = -0.2

In [3]:
def center(l):
    return np.array(l) + H/2

In [4]:
def draw_car(p, l, w, a):
    corners = [[-l/2,w/2], [l/2,w/2], [l/2,-w/2], [-l/2,-w/2]]

    for c in corners:
        temp = c[0]*np.cos(a) - c[1]*np.sin(a) + p[0]
        c[1] = -c[0]*np.sin(a) - c[1]*np.cos(a) + p[1]
        c[0] = temp

    pygame.draw.polygon(screen, 1, center(corners))

In [5]:
class Section():
    def __init__(self, x, y, a, b, length):
        self.x = x
        self.y = y
        self.a = a
        self.b = b
        self.length = length
        self.last_car = None

In [6]:
class Car():
    def __init__(self, length, path):
        self.length = length
        self.path = path
        self.i = 0
        self.done = False
        self.next_car = None
        self.find_next_car(0)
        self.distance = 0
        self.position = path[0].get_position(0)
        self.angle = path[0].get_angle(0)
        self.speed = MAX_SPEED
        self.acceleration = 0
            
    def update_acceleration(self):
        self.acceleration = ACCELERATION
        
        if self.next_car:
            space = self.next_car.distance - self.distance - self.length - BUFFER
            if self.speed and space / self.speed < FOLLOWING_TIME:
                self.acceleration = DECELERATION

#         if self.path.blocked:
#             space = self.path.stop_line - self.distance - self.length
#             braking_distance = self.speed**2 / (2 * ACCELERATION)
#             difference = space - braking_distance

#             if 0 < difference < 10:
#                 self.acceleration = DECELERATION

    def update_speed(self):
        self.speed = max(0, min(MAX_SPEED, self.speed + self.acceleration))

    def update_position(self):
        self.distance += self.speed   
        section = self.path[self.i]
        self.position = section.get_position(self.distance)
        self.angle = section.get_angle(self.distance)
    
    def find_next_car(self, n=1):
        next_car = self.next_car
        
        if not next_car or next_car.done or next_car.path[next_car.i] not in self.path:
            self.next_car = None
            
            for section in self.path[self.i+n:]:
                self.next_car = section.last_car
                if self.next_car:
                    return
    
    def update_section(self):
        section = self.path[self.i]

        if self.distance > section.length:
            self.distance -= section.length

            if section.last_car == self:
                section.last_car = None
            if self.i == len(self.path) - 1:
                self.done = True
            else:
                self.i += 1
                self.path[self.i].last_car = self
            
            return True

In [7]:
class Line(Section):
    def get_position(self, s):
        return (self.x + self.a*s, self.y + self.b*s)

    def get_angle(self, s):
        return np.arctan2(-self.b, self.a)

In [8]:
class Curve(Section):
    def __init__(self, x, y, a, b, r):
        super().__init__(x, y, a, b, np.pi*r/2)
        self.r = r

    def get_position(self, s):
        c = self.a*np.pi/2 + self.b*s/self.r
        return (self.x+self.r*np.cos(c), self.y-self.r*np.sin(c))
    
    def get_angle(self, s):
        return (self.a+1)*np.pi/2 + self.b*s/self.r

In [9]:
def update_cars():
    switched = False

    for car in cars:
#         car.update_acceleration()
        car.update_speed()
        car.update_position()

        if car.update_section():
            switched = True
        if car.done:
            cars.remove(car)

    if switched:
        for car in cars:
            car.find_next_car()

In [10]:
def spawn_car():    
    if random.random() < SPAWN_PROBABILITY:
        if random.random() < TRUCK_PROBABILITY:
            length = random.randint(*TRUCK_LENGTH)
        else:
            length = random.randint(*CAR_LENGTH)

        path = random.choice(paths)
        car = Car(length, path)
        cars.append(car)
        path[0].last_car = car

In [11]:
pygame.init()
screen = pygame.display.set_mode((H,H))
clock = pygame.time.Clock()

L = 50
B = 200

cars = []
sections = [Line(L/2, H/2, 0, -1, H/2-L-B),
            Line(L/2, L+B, 0, -1, B),
            Line(L/2, L, 0, -1, L),
            Line(L/2, 0, 0, -1, L),
            Line(L/2, -L, 0, -1, H/2-L),
            Curve(L, L, 2, -1, L/2),
            Curve(0, 0, 0, 1, L/2)]

for _ in range(3):
    for s in sections[-7:]:
        if isinstance(s, Line):
            sections.append(Line(s.y,-s.x,s.b,-s.a,s.length))
        else:
            sections.append(Curve(s.y,-s.x,s.a+1,s.b,s.r))

l = [[0,1,2,3,4], [0,1,5,25], [0,1,2,6,10,11]]
paths = [[sections[(n+i*7)%28] for n in p] for i in range(4) for p in l]


### game loop ###
for t in itertools.count():
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
    
    spawn_car()
    update_cars()
    
    # draw all graphics
    screen.fill((255, 255, 255))
    for car in cars:
        draw_car(car.position, car.length, CAR_WIDTH, car.angle)
    pygame.display.flip()

    

error: display Surface quit